## Trainer Testing

In [31]:
from util.db import mydb
import tensorflow as tf
from collections import Counter
from extractor import mycursor # also execute extractor first
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from tensorflow.keras import datasets, layers, models
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [32]:
# fetch from DB
sql = ("SELECT id, keyword,rating,raters,viewers,TIMESTAMPDIFF(DAY, created_at, NOW())"
    "FROM wall WHERE keyword IS NOT NULL and created_at > now() - INTERVAL 60 DAY")
mycursor.execute(sql)

walls = mycursor.fetchall()

In [33]:
# Grab X, Y
ids = [x[0] for x in walls]
corpus = [x[1] for x in walls]
statistics = [x[2:5] for x in walls]
age = [x[5] for x in walls]
scaler = MinMaxScaler()
X = CountVectorizer().fit_transform(corpus)
X = TfidfTransformer().fit_transform(X)
X = np.array(X.toarray())
Y = np.array(statistics)
Y = scaler.fit_transform(Y)

In [34]:
# compile
model = models.Sequential([
        tf.keras.layers.Dense(len(ids), activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(24, activation="relu"),
        tf.keras.layers.Dense(10, activation="relu"),
        tf.keras.layers.Dense(3, activation='sigmoid')
    ])
model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer='adam', metrics=['accuracy'])

In [35]:
# train
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4)
history = model.fit(X_train, Y_train, epochs=10, validation_data=(X_test, Y_test))

Epoch 1/10
1/1 [==============================] - 0s 498ms/step - loss: 0.1794 - accuracy: 0.0000e+00 - val_loss: 0.1140 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 15ms/step - loss: 0.1790 - accuracy: 0.6667 - val_loss: 0.1143 - val_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 0s 16ms/step - loss: 0.1787 - accuracy: 0.6667 - val_loss: 0.1145 - val_accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 0s 16ms/step - loss: 0.1784 - accuracy: 0.6667 - val_loss: 0.1146 - val_accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 0s 16ms/step - loss: 0.1780 - accuracy: 0.6667 - val_loss: 0.1148 - val_accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 0s 17ms/step - loss: 0.1776 - accuracy: 0.6667 - val_loss: 0.1150 - val_accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 0s 16ms/step - loss: 0.1773 - accuracy: 0.6667 - val_loss: 0.1152 - val_accuracy: 0.000

In [36]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 5)                 2780      
_________________________________________________________________
dense_11 (Dense)             (None, 32)                192       
_________________________________________________________________
dense_12 (Dense)             (None, 24)                792       
_________________________________________________________________
dense_13 (Dense)             (None, 10)                250       
_________________________________________________________________
dense_14 (Dense)             (None, 3)                 33        
Total params: 4,047
Trainable params: 4,047
Non-trainable params: 0
_________________________________________________________________


In [43]:
predicted = scaler.inverse_transform(model.predict(X))
# push to mysql
sql = "UPDATE wall SET predicted_rating = %s, predicted_raters=%s, predicted_viewers=%s, predicted_score=%s  WHERE id=%s"
for k, v in enumerate(predicted.tolist()):
    mycursor.execute(sql, [*v,v[0]*v[2]/(age[k]+1), ids[k]])
mydb.commit()